In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate

In [ ]:
# Load training data structures
# -----------------------------
with open("D:\Study\Ai_Music_Composer\project\data\generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

note_to_idx = data["note_to_idx"]  # Dictionary mapping note -> index
idx_to_note = data["idx_to_note"]  # Dictionary mapping index -> note

SEQ_LEN = 50
num_moods = 4
num_notes = len(note_to_idx)  # Should be 128 in your case

In [ ]:
# Define Model
# -----------------------------
# Inputs
note_input = tf.keras.layers.Input(shape=(SEQ_LEN, 1), name="note_input")
mood_input = tf.keras.layers.Input(shape=(1,), name="mood_input")

# Note embedding
note_emb = tf.keras.layers.Embedding(
    input_dim=num_notes,
    output_dim=100
)(tf.keras.layers.Flatten()(note_input))

# Mood embedding
mood_emb = tf.keras.layers.Embedding(
    input_dim=num_moods,
    output_dim=10
)(tf.keras.layers.Flatten()(mood_input))  # Flatten to remove extra dimension
mood_emb_repeat = tf.keras.layers.RepeatVector(SEQ_LEN)(mood_emb)

# Combine
x = tf.keras.layers.Concatenate()([note_emb, mood_emb_repeat])

# LSTM layers
x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
x = tf.keras.layers.LSTM(256)(x)

# Output
output = tf.keras.layers.Dense(num_notes, activation="softmax")(x)

# Model
model = tf.keras.Model(inputs=[note_input, mood_input], outputs=output)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
# --------------------------
# LOAD TRAINED WEIGHTS
# --------------------------
model.load_weights("D:\Study\Ai_Music_Composer\project\models\music_generator.h5")  # Make sure this exists

In [ ]:
# -----------------------------
# Function to Generate Music
# -----------------------------
def generate_music(model, seed_sequence, mood, length=100):
    generated = seed_sequence.copy()
    for _ in range(length):
        input_seq = np.array(generated[-SEQ_LEN:]).reshape(1, SEQ_LEN, 1)
        input_mood = np.array([mood]).reshape(1, 1)

        preds = model.predict([input_seq, input_mood], verbose=0)
        next_note = np.argmax(preds, axis=-1)[0]
        generated.append(next_note)

    return generated

In [ ]:
# Ask user for mood
# -----------------------------
mood_map = {
    "happy": 0,
    "sad": 1,
    "angry": 2,
    "relaxed": 3
}

mood_input_user = input("Enter mood (happy, sad, angry, relaxed): ").strip().lower()
if mood_input_user not in mood_map:
    raise ValueError("Invalid mood. Choose from: happy, sad, angry, relaxed")

mood_value = mood_map[mood_input_user]

In [ ]:
# Example seed sequence: 50 zeros (could be replaced with real seed)
seed_sequence = [0] * SEQ_LEN

# Generate music
generated_notes = generate_music(model, seed_sequence, mood_value, length=100)

# Convert notes back to readable format
generated_notes_readable = [idx_to_note[n] for n in generated_notes]
print("Generated notes:", generated_notes_readable)